#  CogniVue: Training

** FULLY COMPATIBLE with your preprocessing notebook!**

This notebook:
-  Loads data from single `.pkl` files per split
-  Handles variable channel count (typically 58 channels)
-  Correctly transposes X from `(n_ch, 256)` to `(256, n_ch)`
-  Robust checkpointing and error handling
-  Resume training from interruptions
-  



##  1. Install Dependencies

##  2. Configuration

In [1]:
# =====================================================
# MODEL HYPERPARAMETERS - IMPROVED FOR GENERALIZATION
# =====================================================

D_MODEL = 256
NUM_LAYERS = 4  # REDUCED from 6 to prevent overfitting
NUM_HEADS = 8
FF_DIM = 512  # REDUCED from 1024 to prevent overfitting
DROPOUT = 0.25  # INCREASED from 0.15 for better regularization

BANDPOWER_HIDDEN_DIM = 128
BANDPOWER_OUTPUT_DIM = 128
TASK_EMBEDDING_DIM = 16

# =====================================================
# TRAINING HYPERPARAMETERS - IMPROVED
# =====================================================

EPOCHS = 100
INITIAL_LR = 5e-5  # REDUCED from 1e-4 for more stable training
WARMUP_EPOCHS = 5  # REDUCED from 10 for faster convergence
WEIGHT_DECAY = 0.02  # INCREASED from 0.01 for stronger regularization
GRADIENT_CLIP_NORM = 0.5  # REDUCED from 1.0 for more aggressive clipping

SAVE_CHECKPOINT_EVERY = 2
EARLY_STOPPING_PATIENCE = 20  # INCREASED from 15 to allow more training

print(f"\nConfiguration:")
print(f"  Model: {NUM_LAYERS} layers, {NUM_HEADS} heads, D_MODEL={D_MODEL}")
print(f"  FF_DIM={FF_DIM}, DROPOUT={DROPOUT}")
print(f"  Training: {EPOCHS} epochs, LR={INITIAL_LR}")
print(f"  Weight Decay: {WEIGHT_DECAY}, Gradient Clip: {GRADIENT_CLIP_NORM}")
print(f"  Checkpointing: every {SAVE_CHECKPOINT_EVERY} epochs")

SyntaxError: invalid character '§' (U+00A7) (2737970318.py, line 28)

##  3. TPU Initialization

In [ ]:
import tensorflow as tf

print("=" * 70)
print("🚀 GPU T4 x2 INITIALIZATION")
print("=" * 70)

print(f"\nTensorFlow version: {tf.__version__}")

# Create GPU strategy
print("\n📊 Creating GPU strategy...")
strategy = tf.distribute.MirroredStrategy()

num_replicas = strategy.num_replicas_in_sync
print(f"✅ Strategy: {strategy.__class__.__name__}")
print(f"   GPUs detected: {num_replicas}")

# List GPUs
gpus = tf.config.list_physical_devices('GPU')
print(f"\n🎮 GPU devices:")
for i, gpu in enumerate(gpus):
    print(f"   {i+1}. {gpu.name}")

# Batch size configuration
BATCH_SIZE_PER_REPLICA = 32  # Reduce to 16 if you get OOM errors
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * num_replicas

print(f"\n📦 Batch Configuration:")
print(f"   Per-GPU batch: {BATCH_SIZE_PER_REPLICA}")
print(f"   Global batch: {BATCH_SIZE}")

print("\n" + "=" * 70)
print("✅ GPU READY FOR TRAINING")
print("=" * 70)

# Make variables global
globals()['strategy'] = strategy
globals()['BATCH_SIZE'] = BATCH_SIZE

print("\n💡 Next: Create model in strategy scope")


In [ ]:
# ============================================================
# AUTO-SAVE CALLBACK (Add this BEFORE training section)
# ============================================================

class AutoSaveCallback(tf.keras.callbacks.Callback):
    """Custom callback to create marker files for auto-committing"""
    
    def on_epoch_end(self, epoch, logs=None):
        # Create a marker file every epoch
        # Kaggle auto-commits when new files appear
        marker_path = f"/kaggle/working/progress_epoch_{epoch+1}.txt"
        with open(marker_path, 'w') as f:
            f.write(f"Completed epoch {epoch+1}/{EPOCHS}\n")
            f.write(f"Loss: {logs.get('loss', 0):.4f}\n")
            f.write(f"Val Loss: {logs.get('val_loss', 0):.4f}\n")
            f.write(f"Timestamp: {datetime.now()}\n")
        
        print(f"   Progress saved: progress_epoch_{epoch+1}.txt")

print(" Auto-save callback ready!")

##  4. Data Loading (Compatible with Preprocessing)

In [ ]:
def load_preprocessed_data(split='train'):
    """
    Load preprocessed .pkl file (matches preprocessing output format).
    
    Returns:
        Tuple of (X, y, metadata) where:
        X = (X_eeg, X_bp, X_task)
        y = (y_channel, y_region, y_band, y_state)
        metadata = dict with num_channels, etc.
    """
    pkl_path = os.path.join(DATA_INPUT_DIR, split, f"{split}_data.pkl")
    
    print(f"\n Loading {split} data from: {pkl_path}")
    
    if not os.path.exists(pkl_path):
        print(f"   File not found!")
        print(f"   Check that dataset is attached and DATASET_NAME is correct")
        return None
    
    # Load pickle file
    try:
        with open(pkl_path, 'rb') as f:
            samples = pickle.load(f)
        
        print(f"   Loaded {len(samples):,} samples")
        
        if len(samples) == 0:
            print(f"   No samples in file!")
            return None
        
        # Inspect first sample to get dimensions
        sample = samples[0]
        X_shape = sample['X'].shape  # Should be (n_channels, 256)
        bp_shape = sample['bp'].shape  # Should be (n_channels, 5)
        
        num_channels = X_shape[0]
        
        print(f"\n  Data format:")
        print(f"     X shape: {X_shape} (channels, time)")
        print(f"     bp shape: {bp_shape} (channels, bands)")
        print(f"     Channels: {num_channels}")
        
        # Extract arrays
        print(f"\n   Converting to arrays...")
        
        # X: Transpose from (n_channels, 256) to (256, n_channels)
        X_eeg = np.array([s['X'].T for s in samples], dtype=np.float32)
        
        # bp: Flatten from (n_channels, 5) to (n_channels*5,)
        X_bp = np.array([s['bp'].flatten() for s in samples], dtype=np.float32)
        
        # task_idx
        X_task = np.array([s['task_idx'] for s in samples], dtype=np.int32)
        
        # Labels
        y_channel = np.array([s['y_channel'] for s in samples], dtype=np.int32)
        y_region = np.array([s['y_region'] for s in samples], dtype=np.int32)
        y_band = np.array([s['y_band'] for s in samples], dtype=np.int32)
        y_state = np.array([s['y_state'] for s in samples], dtype=np.int32)
        
        print(f"   Final shapes:")
        print(f"     X_eeg: {X_eeg.shape} (N, time, channels)")
        print(f"     X_bp: {X_bp.shape} (N, features)")
        print(f"     X_task: {X_task.shape}")
        print(f"     Labels: {y_channel.shape} each")
        
        metadata = {
            'num_channels': num_channels,
            'num_samples': len(samples),
            'bandpower_dim': X_bp.shape[1]
        }
        
        return (X_eeg, X_bp, X_task), (y_channel, y_region, y_band, y_state), metadata
        
    except Exception as e:
        print(f"  ❌ Error loading data: {e}")
        import traceback
        traceback.print_exc()
        return None

print(" Data loading function defined")

##  5. Model Architecture (Flexible Channels)

In [2]:
def create_model(num_channels, bandpower_input_dim, num_output_channels):
    """
    Create EEG Transformer with improved regularization and stability.
    """
    # Inputs
    eeg_input = tf.keras.Input(shape=(WINDOW_SIZE_SAMPLES, num_channels), name='eeg')
    bp_input = tf.keras.Input(shape=(bandpower_input_dim,), name='bp')
    task_input = tf.keras.Input(shape=(1,), dtype='int32', name='task')
    
    # ==================== EEG STREAM ====================
    x = tf.keras.layers.Dense(D_MODEL, name='eeg_projection')(eeg_input)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6, name='eeg_ln')(x)  # ADDED
    
    # Positional encoding
    positions = tf.range(start=0, limit=WINDOW_SIZE_SAMPLES, delta=1)
    pos_emb = tf.keras.layers.Embedding(
        input_dim=WINDOW_SIZE_SAMPLES,
        output_dim=D_MODEL,
        name='positional_embedding'
    )(positions)
    x = x + pos_emb
    x = tf.keras.layers.Dropout(DROPOUT, name='pos_dropout')(x)  # ADDED
    
    # Transformer layers
    for i in range(NUM_LAYERS):
        # Pre-LayerNorm architecture (better stability)
        x_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6, name=f'ln_attn_pre_{i}')(x)
        
        attn = tf.keras.layers.MultiHeadAttention(
            num_heads=NUM_HEADS,
            key_dim=D_MODEL // NUM_HEADS,
            dropout=DROPOUT,
            name=f'mha_{i}'
        )(x_norm, x_norm)
        
        attn = tf.keras.layers.Dropout(DROPOUT, name=f'attn_dropout_{i}')(attn)  # ADDED
        x = tf.keras.layers.Add(name=f'add_attn_{i}')([x, attn])
        
        # FFN with Pre-LayerNorm
        x_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6, name=f'ln_ffn_pre_{i}')(x)
        
        ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(FF_DIM, activation='gelu', name=f'ffn_dense1_{i}'),  # CHANGED to GELU
            tf.keras.layers.Dropout(DROPOUT, name=f'ffn_dropout1_{i}'),
            tf.keras.layers.Dense(D_MODEL, name=f'ffn_dense2_{i}'),
            tf.keras.layers.Dropout(DROPOUT, name=f'ffn_dropout2_{i}')
        ], name=f'ffn_{i}')
        
        ffn_out = ffn(x_norm)
        x = tf.keras.layers.Add(name=f'add_ffn_{i}')([x, ffn_out])
    
    # Final LayerNorm
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6, name='final_ln')(x)  # ADDED
    eeg_emb = tf.keras.layers.GlobalAveragePooling1D(name='eeg_pool')(x)
    
    # ==================== BANDPOWER STREAM ====================
    bp_x = tf.keras.layers.BatchNormalization(name='bp_bn')(bp_input)  # ADDED
    bp_x = tf.keras.layers.Dense(BANDPOWER_HIDDEN_DIM, activation='gelu', name='bp_hidden')(bp_x)
    bp_x = tf.keras.layers.Dropout(DROPOUT, name='bp_dropout1')(bp_x)  # ADDED
    bp_emb = tf.keras.layers.Dense(BANDPOWER_OUTPUT_DIM, activation='gelu', name='bp_output')(bp_x)
    bp_emb = tf.keras.layers.Dropout(DROPOUT, name='bp_dropout2')(bp_emb)  # ADDED
    
    # ==================== TASK STREAM ====================
    task_emb = tf.keras.layers.Embedding(NUM_TASKS, TASK_EMBEDDING_DIM, name='task_emb')(task_input)
    task_emb = tf.keras.layers.Flatten(name='task_flatten')(task_emb)
    task_emb = tf.keras.layers.Dropout(DROPOUT, name='task_dropout')(task_emb)  # ADDED
    
    # ==================== FUSION ====================
    fused = tf.keras.layers.Concatenate(name='fusion')([eeg_emb, bp_emb, task_emb])
    fused = tf.keras.layers.BatchNormalization(name='fusion_bn')(fused)  # ADDED
    fused = tf.keras.layers.Dropout(DROPOUT, name='fusion_dropout')(fused)  # ADDED
    
    # ==================== MULTI-TASK HEADS ====================
    out_channel = tf.keras.layers.Dense(num_output_channels, name='channel')(fused)
    out_region = tf.keras.layers.Dense(NUM_OUTPUT_REGIONS, name='region')(fused)
    out_band = tf.keras.layers.Dense(NUM_OUTPUT_BANDS, name='band')(fused)
    out_state = tf.keras.layers.Dense(NUM_OUTPUT_STATES, name='state')(fused)
    
    model = tf.keras.Model(
        inputs=[eeg_input, bp_input, task_input],
        outputs={
            'channel': out_channel,
            'region': out_region,
            'band': out_band,
            'state': out_state
        },
        name='CogniVue_Transformer'
    )
    
    return model

print("✅ Improved model architecture defined")

✅ Improved model architecture defined


##  6. Learning Rate Schedule

In [3]:
import tensorflow as tf
import numpy as np

class WarmupCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, warmup_steps, total_steps):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
    
    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        total_steps = tf.cast(self.total_steps, tf.float32)
        
        warmup_lr = (step / warmup_steps) * self.initial_learning_rate
        
        decay_steps = total_steps - warmup_steps
        decay_step = step - warmup_steps
        cosine_decay = 0.5 * (1 + tf.cos(tf.constant(np.pi) * decay_step / decay_steps))
        decay_lr = self.initial_learning_rate * cosine_decay
        
        return tf.cond(
            step < warmup_steps,
            lambda: warmup_lr,
            lambda: decay_lr
        )
    
    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "warmup_steps": self.warmup_steps,
            "total_steps": self.total_steps,
        }

print("LR schedule defined")

NameError: name 'tf' is not defined

##  7. Data Pipeline & Callbacks

In [ ]:
import tensorflow as tf
import time

def augment_eeg(eeg, training=True):
    """Apply data augmentation to EEG signals during training"""
    if not training:
        return eeg
    
    # Random noise addition (5% of samples)
    if tf.random.uniform([]) < 0.05:
        noise = tf.random.normal(tf.shape(eeg), mean=0.0, stddev=0.02)
        eeg = eeg + noise
    
    # Random amplitude scaling (10% of samples)
    if tf.random.uniform([]) < 0.1:
        scale = tf.random.uniform([], minval=0.95, maxval=1.05)
        eeg = eeg * scale
    
    return eeg


def create_tf_dataset(X, y, is_train=True):
    """Create TF dataset with augmentation and improved batching"""
    dataset = tf.data.Dataset.from_tensor_slices((
        {'eeg': X[0], 'bp': X[1], 'task': X[2]},
        {'channel': y[0], 'region': y[1], 'band': y[2], 'state': y[3]}
    ))
    
    if is_train:
        # Shuffle with larger buffer
        dataset = dataset.shuffle(buffer_size=min(50000, len(X[0])))
        
        # Apply augmentation
        def augment_fn(inputs, labels):
            inputs['eeg'] = augment_eeg(inputs['eeg'], training=True)
            return inputs, labels
        
        dataset = dataset.map(augment_fn, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Batch and prefetch
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset


class PeriodicCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_freq=5):
        super().__init__()
        self.save_freq = save_freq
    
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            filepath = os.path.join(CHECKPOINT_DIR, f"checkpoint_epoch_{epoch+1:03d}.keras")
            self.model.save(filepath)
            print(f"\n   Saved checkpoint: {os.path.basename(filepath)}")


class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate
        if hasattr(lr, '__call__'):
            lr_value = lr(self.model.optimizer.iterations)
        else:
            lr_value = lr
        lr_float = float(tf.keras.backend.get_value(lr_value))
        if logs is not None:
            logs['learning_rate'] = lr_float
        print(f"\n   LR = {lr_float:.6f}")


class ProgressLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.epoch_start = None
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()
        print(f"\n{'='*70}")
        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"{'='*70}")
    
    def on_epoch_end(self, epoch, logs=None):
        elapsed = time.time() - self.epoch_start
        print(f"\nEpoch {epoch+1} done in {elapsed:.1f}s")
        if logs:
            print(f"   Loss: {logs.get('loss', 0):.4f} | Val Loss: {logs.get('val_loss', 0):.4f}")
            print(f"   Region Acc: {logs.get('region_accuracy', 0):.4f} | Val: {logs.get('val_region_accuracy', 0):.4f}")
            
            # Calculate gap
            gap = logs.get('loss', 0) - logs.get('val_loss', 0)
            if gap < -0.3:
                print(f"   WARNING: Overfitting detected (gap: {gap:.4f})")
        print(f"{'='*70}\n")


class AdaptiveLRCallback(tf.keras.callbacks.Callback):
    """Reduce LR when validation loss plateaus"""
    def __init__(self, patience=5, factor=0.5, min_lr=1e-6):
        super().__init__()
        self.patience = patience
        self.factor = factor
        self.min_lr = min_lr
        self.best_val_loss = float('inf')
        self.wait = 0
    
    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                old_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
                if old_lr > self.min_lr:
                    new_lr = max(old_lr * self.factor, self.min_lr)
                    tf.keras.backend.set_value(self.model.optimizer.learning_rate, new_lr)
                    print(f"\n   Reducing LR: {old_lr:.6f} -> {new_lr:.6f}")
                    self.wait = 0


print("Pipeline & callbacks defined")

##  8. Main Training Function

In [ ]:
from datetime import datetime
import numpy as np

def train_cognivue():
    print("\n" + "="*70)
    print("CogniVue Training Pipeline (IMPROVED)")
    print("="*70)
    
    # Load ONLY training data
    print("\nLoading data...")
    train_result = load_preprocessed_data('train')
    
    if train_result is None:
        print("\nData loading failed!")
        return None
    
    train_data, train_labels, train_meta = train_result
    
    # Split train/val
    print("\nSplitting data into train/val...")
    num_samples = len(train_data[0])
    indices = np.arange(num_samples)
    np.random.seed(42)
    np.random.shuffle(indices)
    
    split_idx = int(0.8 * num_samples)
    train_idx = indices[:split_idx]
    val_idx = indices[split_idx:]
    
    train_X = (train_data[0][train_idx], train_data[1][train_idx], train_data[2][train_idx])
    val_X = (train_data[0][val_idx], train_data[1][val_idx], train_data[2][val_idx])
    train_y = (train_labels[0][train_idx], train_labels[1][train_idx], 
               train_labels[2][train_idx], train_labels[3][train_idx])
    val_y = (train_labels[0][val_idx], train_labels[1][val_idx], 
             train_labels[2][val_idx], train_labels[3][val_idx])
    
    print(f"   Train samples: {len(train_idx):,}")
    print(f"   Val samples: {len(val_idx):,}")
    
    # Get dimensions
    NUM_CHANNELS = train_meta['num_channels']
    BANDPOWER_INPUT_DIM = train_meta['bandpower_dim']
    NUM_OUTPUT_CHANNELS = train_labels[0].max() + 1
    
    print(f"\nModel dimensions:")
    print(f"   Input channels: {NUM_CHANNELS}")
    print(f"   Bandpower dim: {BANDPOWER_INPUT_DIM}")
    print(f"   Output channels: {NUM_OUTPUT_CHANNELS}")
    
    # Create datasets
    print(f"\nCreating TF datasets...")
    train_ds = create_tf_dataset(train_X, train_y, is_train=True)
    val_ds = create_tf_dataset(val_X, val_y, is_train=False)
    
    steps_per_epoch = len(train_idx) // BATCH_SIZE
    total_steps = steps_per_epoch * EPOCHS
    warmup_steps = steps_per_epoch * WARMUP_EPOCHS
    
    print(f"   Steps/epoch: {steps_per_epoch:,}")
    print(f"   Total steps: {total_steps:,}")
    
    # Build model
    print(f"\nBuilding model...")
    
    with strategy.scope():
        model = create_model(NUM_CHANNELS, BANDPOWER_INPUT_DIM, NUM_OUTPUT_CHANNELS)
        print(f"   Parameters: {model.count_params():,}")
        
        lr_schedule = WarmupCosineDecay(INITIAL_LR, warmup_steps, total_steps)
        
        optimizer = tf.keras.optimizers.AdamW(
            learning_rate=lr_schedule,
            weight_decay=WEIGHT_DECAY,
            clipnorm=GRADIENT_CLIP_NORM
        )
        
        # IMPROVED: Balanced loss weights
        loss_weights = {
            'channel': 0.35,
            'region': 0.40,
            'band': 0.15,
            'state': 0.10
        }
        
        # IMPROVED: Add label smoothing
        model.compile(
            optimizer=optimizer,
            loss={
                'channel': tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=True, label_smoothing=0.1),
                'region': tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=True, label_smoothing=0.1),
                'band': tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=True, label_smoothing=0.1),
                'state': tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=True, label_smoothing=0.1)
            },
            loss_weights=loss_weights,
            metrics={
                'channel': ['accuracy'],
                'region': ['accuracy'],
                'band': ['accuracy'],
                'state': ['accuracy']
            }
        )
        
        print(f"   Compiled with label smoothing")
    
    # Callbacks
    callbacks = [
        ProgressLogger(),
        LearningRateLogger(),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(CHECKPOINT_DIR, 'best_model.keras'),
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        PeriodicCheckpoint(save_freq=SAVE_CHECKPOINT_EVERY),
        AdaptiveLRCallback(patience=5, factor=0.5, min_lr=1e-6),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=EARLY_STOPPING_PATIENCE,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.TensorBoard(log_dir=LOGS_DIR)
    ]
    
    # Train
    print(f"\nStarting training...")
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    
    try:
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds,
            callbacks=callbacks,
            verbose=1
        )
        
        print("\n" + "="*70)
        print("TRAINING COMPLETE!")
        print("="*70)
        print(f"\nSaved to: {CHECKPOINT_DIR}")
        
        model.save(os.path.join(CHECKPOINT_DIR, 'final_model.keras'))
        print(f"   - final_model.keras")
        print(f"   - best_model.keras")
        
        return history
        
    except KeyboardInterrupt:
        print("\nTraining interrupted!")
        model.save(os.path.join(CHECKPOINT_DIR, 'interrupted.keras'))
        print(f"   Saved: interrupted.keras")
        return None
        
    except Exception as e:
        print(f"\nTraining failed: {e}")
        import traceback
        traceback.print_exc()
        return None

print("Training function ready")

##  9. Run Training

In [ ]:
history = train_cognivue()

if history:
    print("\n Training Summary:")
    print(f"   Best val_loss: {min(history.history['val_loss']):.4f}")
    print(f"   Final region acc: {history.history['region_accuracy'][-1]:.4f}")
    print(f"\n Next: Click 'Save Version' to commit checkpoints!")
else:
    print("\n⚠️ Training did not complete successfully")

10: Save Training Results & Metrics

In [ ]:
if history:
    print("\n Saving training results...")
    
    # Convert history to JSON-serializable format
    history_dict = {
        key: [float(val) for val in values] 
        for key, values in history.history.items()
    }
    
    # Save training history
    history_path = os.path.join(RESULTS_DIR, 'training_history.json')
    with open(history_path, 'w') as f:
        json.dump(history_dict, f, indent=2)
    print(f"   Saved: training_history.json")
    
    # Get model dimensions from loaded data
    train_result = load_preprocessed_data('train')
    if train_result:
        _, _, train_meta = train_result
        NUM_CHANNELS_USED = train_meta['num_channels']
    else:
        NUM_CHANNELS_USED = "unknown"
    
    # Save training configuration
    config = {
        'model_architecture': {
            'name': 'CogniVue_Transformer',
            'num_input_channels': NUM_CHANNELS_USED,
            'd_model': D_MODEL,
            'num_layers': NUM_LAYERS,
            'num_heads': NUM_HEADS,
            'ff_dim': FF_DIM,
            'dropout': DROPOUT,
            'window_size': WINDOW_SIZE_SAMPLES
        },
        'training_params': {
            'epochs_trained': len(history.history['loss']),
            'total_epochs': EPOCHS,
            'batch_size': BATCH_SIZE,
            'initial_lr': INITIAL_LR,
            'warmup_epochs': WARMUP_EPOCHS,
            'weight_decay': WEIGHT_DECAY,
            'gradient_clip_norm': GRADIENT_CLIP_NORM
        },
        'output_tasks': {
            'num_output_channels': NUM_OUTPUT_REGIONS,
            'num_regions': NUM_OUTPUT_REGIONS,
            'num_bands': NUM_OUTPUT_BANDS,
            'num_states': NUM_OUTPUT_STATES
        },
        'final_metrics': {
            'best_val_loss': float(min(history.history['val_loss'])),
            'final_train_loss': float(history.history['loss'][-1]),
            'final_val_loss': float(history.history['val_loss'][-1]),
            'final_region_accuracy': float(history.history['region_accuracy'][-1]),
            'final_val_region_accuracy': float(history.history['val_region_accuracy'][-1])
        },
        'training_info': {
            'completed_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'tensorflow_version': tf.__version__,
            'accelerator': 'TPU' if 'TPU' in str(strategy.__class__) else 'CPU/GPU'
        }
    }
    
    config_path = os.path.join(RESULTS_DIR, 'training_config.json')
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"   Saved: training_config.json")
    
    # Create a summary markdown file
    summary_md = f"""# CogniVue Training Summary

**Training Completed:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Model Architecture
- **Model:** CogniVue Transformer
- **Input Channels:** {NUM_CHANNELS_USED}
- **Model Dimension:** {D_MODEL}
- **Transformer Layers:** {NUM_LAYERS}
- **Attention Heads:** {NUM_HEADS}
- **Feedforward Dim:** {FF_DIM}
- **Dropout:** {DROPOUT}

## Training Configuration
- **Epochs:** {len(history.history['loss'])}/{EPOCHS}
- **Batch Size:** {BATCH_SIZE}
- **Initial LR:** {INITIAL_LR}
- **Warmup Epochs:** {WARMUP_EPOCHS}
- **Weight Decay:** {WEIGHT_DECAY}

## Final Performance
- **Best Val Loss:** {min(history.history['val_loss']):.4f}
- **Final Train Loss:** {history.history['loss'][-1]:.4f}
- **Final Val Loss:** {history.history['val_loss'][-1]:.4f}
- **Final Region Accuracy:** {history.history['region_accuracy'][-1]:.4f}
- **Final Val Region Accuracy:** {history.history['val_region_accuracy'][-1]:.4f}

## Output Files
- `checkpoints/best_model.keras` - Best model weights
- `checkpoints/final_model.keras` - Final model weights
- `checkpoints/checkpoint_epoch_*.keras` - Periodic checkpoints
- `results/training_history.json` - Loss and metrics per epoch
- `results/training_config.json` - Full configuration
- `logs/` - TensorBoard logs
"""
    
    summary_path = os.path.join(RESULTS_DIR, 'TRAINING_SUMMARY.md')
    with open(summary_path, 'w') as f:
        f.write(summary_md)
    print(f"   Saved: TRAINING_SUMMARY.md")
    
    print("\n All results saved!")
    print(f"\n Saved files:")
    print(f"   {RESULTS_DIR}/")
    print(f"   ├── training_history.json")
    print(f"   ├── training_config.json")
    print(f"   └── TRAINING_SUMMARY.md")
    print(f"\n   {CHECKPOINT_DIR}/")
    print(f"   ├── best_model.keras")
    print(f"   ├── final_model.keras")
    print(f"   └── checkpoint_epoch_*.keras")
    
else:
    print("\n⚠️ No training history to save")




11: Package & Download All Outputs

In [4]:

# ============================================================
# Section 11: Package & Download All Outputs
# ============================================================

import zipfile
from pathlib import Path

print("\n Creating download package...")
print("=" * 70)

# Create zip filename with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f"cognivue_training_outputs_{timestamp}.zip"
zip_path = os.path.join(WORKING_DIR, zip_filename)

try:
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        
        # Add all files from results directory
        print("\n Adding results...")
        if os.path.exists(RESULTS_DIR):
            for file in os.listdir(RESULTS_DIR):
                file_path = os.path.join(RESULTS_DIR, file)
                if os.path.isfile(file_path):
                    arcname = os.path.join('results', file)
                    zipf.write(file_path, arcname)
                    print(f"   {file}")
        
        # Add all checkpoint files
        print("\n🔖 Adding checkpoints...")
        if os.path.exists(CHECKPOINT_DIR):
            for file in os.listdir(CHECKPOINT_DIR):
                file_path = os.path.join(CHECKPOINT_DIR, file)
                if os.path.isfile(file_path):
                    arcname = os.path.join('checkpoints', file)
                    zipf.write(file_path, arcname)
                    file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
                    print(f"   {file} ({file_size_mb:.1f} MB)")
        
        # Add TensorBoard logs (optional - can be large)
        print("\n Adding TensorBoard logs...")
        if os.path.exists(LOGS_DIR):
            log_count = 0
            for root, dirs, files in os.walk(LOGS_DIR):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.join('logs', os.path.relpath(file_path, LOGS_DIR))
                    zipf.write(file_path, arcname)
                    log_count += 1
            print(f"   Added {log_count} log files")
    
    # Get final zip size
    zip_size_mb = os.path.getsize(zip_path) / (1024 * 1024)
    
    print("\n" + "=" * 70)
    print(" PACKAGE CREATED SUCCESSFULLY!")
    print("=" * 70)
    print(f"\n Zip file: {zip_filename}")
    print(f" Size: {zip_size_mb:.1f} MB")
    print(f" Location: {zip_path}")
    
    print("\n To download:")
    print("   1. Go to the 'Output' tab (top right)")
    print("   2. Click 'Save Version' to commit outputs")
    print(f"   3. Download '{zip_filename}'")
    print("\n Or click the download icon next to the file in the Output tab")
    
    # List contents
    print("\n Package contents:")
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        file_list = zipf.namelist()
        print(f"   Total files: {len(file_list)}")
        print("\n   Structure:")
        print("   ├── results/")
        print("   │   ├── training_history.json")
        print("   │   ├── training_config.json")
        print("   │   └── TRAINING_SUMMARY.md")
        print("   ├── checkpoints/")
        print("   │   ├── best_model.keras")
        print("   │   ├── final_model.keras")
        print("   │   └── checkpoint_epoch_*.keras")
        print("   └── logs/")
        print("       └── TensorBoard logs")
    
    print("\n" + "=" * 70)
    
except Exception as e:
    print(f"\n❌ Error creating zip: {e}")
    import traceback
    traceback.print_exc()


 Creating download package...


NameError: name 'datetime' is not defined

In [ ]:
import os
print(os.listdir('/kaggle/working/'))

In [ ]:
import os
import glob
from IPython.display import FileLink, display

# 1. Search for any zip files starting with 'cognivue_training_outputs'
zip_files = glob.glob('/kaggle/working/cognivue_training_outputs_*.zip')

if zip_files:
    # 2. Sort by newest (incase there are multiple)
    latest_zip = max(zip_files, key=os.path.getctime)
    relative_path = os.path.basename(latest_zip)
    
    print("=" * 50)
    print(f"✅ LATEST PACKAGE FOUND: {relative_path}")
    print("Click the link below to download your results:")
    print("=" * 50)
    
    # 3. Display the dynamic link
    display(FileLink(relative_path))
else:
    print("❌ No output zip files found in /kaggle/working/")
    print("Current directory contents:", os.listdir('/kaggle/working/'))